In [2]:
import re

import keras
import numpy as np
import pandas as pd
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, LSTM

### Обработка данных

In [5]:
data = pd.read_csv("data/animes.csv")
text = ""
for i in data["synopsis"][:250]:
    i = str(i).lower().split("\n")[0]
    text += i
text += data["synopsis"][658].lower().split("\n")[0]

In [6]:
text = text.replace(",", " ")
text = text.replace(":", " ")
text = text.replace("-", " ")
text = text.replace("—", " ")
text = text.replace("―", " ")
text = text.replace("―", " ")
text = text.replace("[", " ")
text = text.replace("]", " ")
text = text.replace("(", " ")
text = text.replace(")", " ")
text = text.replace("\"", " ")
text = text.replace("“", " ")
text = text.replace("”", " ")
text = text.replace("\r", " ")
text = text.replace("☆", " ")
text = text.replace("♪", " ")
text = text.replace("/", " ")
text = text.replace("~", " ")
text = text.replace("&", " ")
text = text.replace("*", " ")
text = text.replace("è", " ")
text = text.replace("é", " ")
text = text.replace("ï", " ")
text = text.replace("ü", " ")
text = text.replace("μ", " ")

text = text.replace("...", ".")
text = text.replace("…", ".")
text = text.replace("?", ".")
text = text.replace("!", ".")
text = text.replace(";", ".")

text = re.sub(r"\s+", " ", text)
text = re.sub(r" \.", ".", text)
text = re.sub(r"\.+", ".", text)

print(text)

following their participation at the inter high the karasuno high school volleyball team attempts to refocus their efforts aiming to conquer the spring tournament instead. music accompanies the path of the human metronome the prodigious pianist kousei arima. but after the passing of his mother saki arima kousei falls into a downward spiral rendering him unable to hear the sound of his own piano. the abyss a gaping chasm stretching down into the depths of the earth filled with mysterious creatures and relics from a time long past. how did it come to be. what lies at the bottom. countless brave individuals known as divers have sought to solve these mysteries of the abyss fearlessly descending into its darkest realms. the best and bravest of the divers the white whistles are hailed as legends by those who remain on the surface. in order for something to be obtained something of equal value must be lost. after helping revive the legendary vampire kiss shot acerola orion heart under blade k

In [7]:
def split_text(ls):
    char_to_index, index_to_char = {}, {}

    for index, char in enumerate(ls):
        char_to_index[char] = index
        index_to_char[index] = char
    return char_to_index, index_to_char

### LSTM

In [126]:
text_chars = sorted(set(text))
char_to_index, index_to_char = split_text(text_chars)
print(char_to_index)
print(index_to_char)

{' ': 0, "'": 1, '.': 2, '0': 3, '1': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11, '9': 12, 'a': 13, 'b': 14, 'c': 15, 'd': 16, 'e': 17, 'f': 18, 'g': 19, 'h': 20, 'i': 21, 'j': 22, 'k': 23, 'l': 24, 'm': 25, 'n': 26, 'o': 27, 'p': 28, 'q': 29, 'r': 30, 's': 31, 't': 32, 'u': 33, 'v': 34, 'w': 35, 'x': 36, 'y': 37, 'z': 38, '’': 39}
{0: ' ', 1: "'", 2: '.', 3: '0', 4: '1', 5: '2', 6: '3', 7: '4', 8: '5', 9: '6', 10: '7', 11: '8', 12: '9', 13: 'a', 14: 'b', 15: 'c', 16: 'd', 17: 'e', 18: 'f', 19: 'g', 20: 'h', 21: 'i', 22: 'j', 23: 'k', 24: 'l', 25: 'm', 26: 'n', 27: 'o', 28: 'p', 29: 'q', 30: 'r', 31: 's', 32: 't', 33: 'u', 34: 'v', 35: 'w', 36: 'x', 37: 'y', 38: 'z', 39: '’'}


In [127]:
chain_length = 20
dataX, dataY = [], []
for sentence in text.split("."):
    for i in range(len(sentence) - chain_length):
        dataX.append(sentence[i:i + chain_length])
        dataY.append(sentence[i + chain_length])

In [128]:
X, Y = np.zeros((len(dataX), chain_length, len(char_to_index))), np.zeros((len(dataX), len(char_to_index)))
for i, sentence in enumerate(dataX):
    for j, char in enumerate(sentence):
        X[i, j, char_to_index[char]] = 1
    Y[i, char_to_index[dataY[i]]] = 1

print(np.argmax(X[0]), index_to_char[np.argmax(X[0])])

18 f


In [129]:
print(X.shape)

(73073, 20, 40)


In [130]:
filepath = "weights/weights-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor="loss", verbose=1, save_best_only=True, mode="min")
callbacks = [checkpoint]

In [131]:
model = keras.Sequential()
model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(Y.shape[1], activation="softmax"))
model.compile(optimizer="adam", loss="categorical_crossentropy")
model.fit(X, Y, epochs=30, callbacks=callbacks)
model.summary()

Epoch 1/30
2282/2284 [============================>.] - ETA: 0s - loss: 2.4669
Epoch 1: loss improved from inf to 2.46679, saving model to weights\weights-01-2.4668.hdf5
2284/2284 [==============================] - 44s 18ms/step - loss: 2.4668
Epoch 2/30
2283/2284 [============================>.] - ETA: 0s - loss: 2.1637
Epoch 2: loss improved from 2.46679 to 2.16364, saving model to weights\weights-02-2.1636.hdf5
2284/2284 [==============================] - 44s 19ms/step - loss: 2.1636
Epoch 3/30
2283/2284 [============================>.] - ETA: 0s - loss: 2.0396
Epoch 3: loss improved from 2.16364 to 2.03960, saving model to weights\weights-03-2.0396.hdf5
2284/2284 [==============================] - 43s 19ms/step - loss: 2.0396
Epoch 4/30
2284/2284 [==============================] - ETA: 0s - loss: 1.9442
Epoch 4: loss improved from 2.03960 to 1.94423, saving model to weights\weights-04-1.9442.hdf5
2284/2284 [==============================] - 43s 19ms/step - loss: 1.9442
Epoch 5/30
2

In [132]:
for pattern in [
    "mai sakurajima is a ",
    "kiyotaka ayanokouji ",
    "nao tomori catches y",
    "saitama has a rather",
    "hirasawa yui a young"
]:
    start_pattern = pattern
    result = pattern
    for _ in range(50):
        x = np.zeros((1, chain_length, len(text_chars)))
        for i, char in enumerate(pattern):
            x[0, i, char_to_index[char]] = 1
        next = index_to_char[np.argmax(model.predict(x))]
        pattern = pattern[1:] + next
        result += next
    print(f"Pattern: {start_pattern}")
    print(f"Prediction: {result}")
    print("-----------------")

1/1 [==============================] - 0s 31ms/step
Pattern: mai sakurajima is a 
Prediction: mai sakurajima is a stapping the second which such as the mories more 
-----------------
1/1 [==============================] - 0s 27ms/step
Pattern: kiyotaka ayanokouji 
Prediction: kiyotaka ayanokouji for the manga which he is an a come tragini akima 
-----------------
1/1 [==============================] - 0s 28ms/step
Pattern: nao tomori catches y
Prediction: nao tomori catches year studen for the magical body him to has enthrea
-----------------
1/1 [==============================] - 0s 25ms/step
Pattern: saitama has a rather
Prediction: saitama has a rather raishu kid a cades of the manga release of the sh
-----------------
1/1 [==============================] - 0s 24ms/step
Pattern: hirasawa yui a young
Prediction: hirasawa yui a young consident them on the shout up non the subjugatio
-----------------


### Markov

In [133]:
chain_length = 20
sequences = [text[i:i + chain_length] for i in range(0, len(text) - chain_length)]
unique_sequences = sorted(set(sequences))

gram_to_index, _ = split_text(unique_sequences)

chain = np.zeros((len(unique_sequences), len(char_to_index)))
for i, seq in enumerate(sequences):
    chain[gram_to_index[seq], char_to_index[text[i + chain_length]]] += 1

for i in range(len(unique_sequences)):
    chain[i] /= chain[i].sum()

In [137]:
index = np.random.randint(0, len(text) - chain_length - 1)
pattern = text[index: index + chain_length]
start_pattern = pattern
result = pattern
for _ in range(50):
    probabilities = chain[gram_to_index[pattern]]
    next = index_to_char[np.random.choice(a=np.arange(len(probabilities)), p=probabilities)]
    result += next
    pattern = pattern[1:] + next
print(f"Pattern: {start_pattern}")
print(f"Prediction: {result}")

Pattern:  singing and decides
Prediction:  singing and decides to take her in with his troupe. having attended m
